In [1]:
import spark_env

spark = spark_env.create_spark_session('jobs')

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
megamart_df = spark.read.format('csv')\
                        .option('header',True)\
                        .option('inferSchema',True)\
                        .load('MegaMart.csv')

In [4]:
megamart_df.show(5)

+--------+-------+----------+----------+----------------+------------+--------+--------------+--------------+------------+
|order_id|user_id|order_date|product_id|product_category|product_name|quantity|price_per_unit|payment_method|order_status|
+--------+-------+----------+----------+----------------+------------+--------+--------------+--------------+------------+
|    1001|   U188|2025-04-20|      P940|         Fashion|    Sneakers|       2|         58.53|        PayPal|   Cancelled|
|    1002|   U062|2025-04-16|      P794|         Fashion|     T-Shirt|       3|         83.76|           UPI|    Returned|
|    1003|   U058|2025-04-18|      P326|         Fashion|  Sunglasses|       2|         78.85|        PayPal|  Processing|
|    1004|   U011|2025-04-10|      P574|         Fashion|  Sunglasses|       5|         46.49|        PayPal|   Delivered|
|    1005|   U003|2025-04-19|      P988|      Home Decor| Photo Frame|       2|         78.61|        PayPal|    Returned|
+--------+------

In [5]:
df = megamart_df.filter(col('product_name') == 'Sunglasses')
df = df.select(
        col('order_id'),
        col('product_name'),
        (col('quantity')*col('price_per_unit')).alias('revenue')
    )
df = df.groupBy(col('product_name')).agg(
    count(col('order_id')).alias('total_orders'),
    round(sum(col('revenue')),2).alias('total_revenue')
)

In [6]:
df.show()

+------------+------------+-------------+
|product_name|total_orders|total_revenue|
+------------+------------+-------------+
|  Sunglasses|          36|      6463.17|
+------------+------------+-------------+

